# Embedding

> Fill in a module description here

In [ ]:
#| default_exp transformer.embedding

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import math

import torch
from torch import nn

### Text Embedding

one for source, one for target

In [ ]:
#| export
class TextEmbedding(nn.Module):
    def __init__(self, vocab_size: int, padding_idx, d_model: int):
        super().__init__()
        self.d_model = d_model
        self.embed = nn.Embedding(vocab_size, d_model, padding_idx=padding_idx)
    
    def forward(self, x):
        # shape(x) = [batch_size x seq_len]
        # shape(embedding) = [batch_size x seq_len x d_model]
        embedding = self.embed(x)
        return embedding * math.sqrt(self.d_model)

In [ ]:
embed = nn.Embedding(5, 4, padding_idx=0)

In [ ]:
embed(torch.tensor([1, 2, 3, 0, 0]))

tensor([[-0.8024,  0.3854, -0.9739,  2.5922],
        [ 3.3359, -1.2493,  0.0466,  1.5786],
        [ 1.7705, -1.1482,  0.0487,  0.8839],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]], grad_fn=<EmbeddingBackward0>)

### Positional Encoding

$P E_{p o s, 2 i}=\sin \left(\frac{p o s}{10000^{\frac{2 i}{d}}}\right)$

$P E_{p o s, 2 i+1}=\cos \left(\frac{p o s}{10000^{\frac{2 i}{d}}}\right)$

- `pos`: the position of a word in a sequence
- `i`: is the index in the word encoding vector

In [ ]:
#| export
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.3, max_seq_len : float = 2000):
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_seq_len, d_model)
        pos = torch.arange(0, max_seq_len).unsqueeze(1).float()
        
        two_i = torch.arange(0, d_model, step=2).float()
        div_term = torch.pow(1000, (two_i/torch.tensor([d_model]))).float()
        
        pe[:, 0::2] = torch.sin(pos/div_term)
        pe[:, 1::2] = torch.cos(pos/div_term)
        
        # add one dim for batch_size
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)
    
    def forward(self, x: torch.Tensor):
        # x is text embedding
        # shape(x) = [batch_size x seq_len x d_model]
        seq_len = x.shape[1]
        
        # extract the position for seq_len
        pe = self.pe[:, :seq_len].detach()
        
        x = x.add(pe)
        
        return self.dropout(x)

In [ ]:
position = PositionalEncoding(8)

In [ ]:
position.pe.shape

torch.Size([1, 2000, 8])

In [ ]:
position.pe

tensor([[[ 0.0000,  1.0000,  0.0000,  ...,  1.0000,  0.0000,  1.0000],
         [ 0.8415,  0.5403,  0.1769,  ...,  0.9995,  0.0056,  1.0000],
         [ 0.9093, -0.4161,  0.3482,  ...,  0.9980,  0.0112,  0.9999],
         ...,
         [-0.8689,  0.4950, -0.1221,  ...,  0.9496, -0.9727,  0.2322],
         [-0.0529,  0.9986, -0.2958,  ...,  0.9392, -0.9713,  0.2377],
         [ 0.8117,  0.5841, -0.4601,  ...,  0.9279, -0.9700,  0.2432]]])